<a href="https://colab.research.google.com/github/szucshey/osm-data-exploration/blob/main/climate/climate.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install netCDF4 basemap folium osm2geojson --quiet

In [ ]:
#Only run when opened in colab
!apt-get install subversion -qq
!svn export https://github.com/szucshey/osm-data-exploration/trunk/climate/datasets
!svn export https://github.com/szucshey/osm-data-exploration/trunk/climate/results

In [2]:
import matplotlib.pyplot as plt
from netCDF4 import Dataset
import xarray as xr
import numpy as np
import PIL
from datetime import datetime, timedelta, date
from mpl_toolkits.basemap import Basemap
import os

/home/szucshey/.local/lib/python3.8/site-packages/xarray/backends/cfgrib_.py:29: UserWarning: Failed to load cfgrib - most likely there is a problem accessing the ecCodes library. Try `import cfgrib` to get the full error message
  warnings.warn(
/usr/lib/python3/dist-packages/requests/__init__.py:89: RequestsDependencyWarning: urllib3 (1.26.12) or chardet (3.0.4) doesn't match a supported version!
  warnings.warn("urllib3 ({}) or chardet ({}) doesn't match a supported "


In [3]:
ds = Dataset("datasets/tg_ens_mean_0.1deg_reg_1965-1979_v25.0e.nc_sliced")

lats = ds.variables['latitude'][:]
lons = ds.variables['longitude'][:]
dates = ds.variables['time'][:]
temp = ds.variables['tg'][:]

lon, lat = np.meshgrid(lons, lats)

In [5]:
start_date = date(1965, 1, 1)

days = 0

for i in range(len(dates)):
  end_date = start_date + timedelta(days=days)
  y, m, d = end_date.strftime("%Y"), end_date.strftime("%m"), end_date.strftime("%d")
  end_date_str = y + "-" + m + "-" + d

  map = Basemap(llcrnrlon=16.06, llcrnrlat=45.65, urcrnrlon=23.02, urcrnrlat=48.63, projection='merc', resolution='i')

  x,y = map(lon, lat)
  map.drawcountries()

  plt.title("Napi átlag hőmérséklet: {0}".format(end_date_str))

  color_scheme = map.pcolormesh(x, y, np.squeeze(temp[i, :, :]), cmap='jet', vmin=-30, vmax=40)
  color_bar = map.colorbar(color_scheme, location='right', pad='5%')
  color_bar.set_label('Napi átlag hőmérséklet (°C)', labelpad=5)
  plt.savefig("results/{0}.jpg".format(days), dpi=300)
  plt.clf()
  days += 1

<Figure size 640x480 with 0 Axes>

In [ ]:
frames = []

for i in range(len(dates)):
  frame = PIL.Image.open("results/{0}.jpg".format(str(i+1)))
  frames.append(frame)

frames[0].save('results/output.gif', 
               format='GIF',
               append_images = frames[1: ], 
               save_all = True,
               duration=20, loop=0)